# Topic 12: Statistical Analysis in R
This notebook covers descriptive statistics, hypothesis testing (t-test, chi-square, ANOVA), correlation, simple linear regression, non-parametric test, and optional `dplyr`/`ggplot2` examples. Each exercise has a **Problem → Student cell → Solution + Interpretation**.

## Exercise 1: Descriptive Statistics
**Problem:** Create a sample dataset of 30 students with `Math`, `Science`, `English` scores. Compute mean, median, mode, variance, standard deviation, range, and show `summary()`.

In [ ]:
# Solution
set.seed(123)
students <- data.frame(
  ID = 1:30,
  Math = round(rnorm(30, mean=70, sd=10)),
  Science = round(rnorm(30, mean=68, sd=12)),
  English = round(rnorm(30, mean=72, sd=8)),
  Gender = sample(c('Male','Female'), 30, replace=TRUE),
  Class = sample(c('A','B','C'), 30, replace=TRUE)
)

# Basic stats
mean_math <- mean(students$Math)
median_math <- median(students$Math)
mode_math <- as.numeric(names(sort(table(students$Math), decreasing=TRUE)[1]))
var_math <- var(students$Math)
sd_math <- sd(students$Math)
range_math <- range(students$Math)

list(mean=mean_math, median=median_math, mode=mode_math, variance=var_math, sd=sd_math, range=range_math)

# Summary of dataset
summary(students)

# Interpretation:
# - mean: average score
# - median: middle value (robust to outliers)
# - mode: most frequent value
# - variance/sd: spread of scores


## Exercise 2: Correlation and Covariance
**Problem:** Compute covariance and Pearson correlation between Math and Science scores. Plot a scatter plot with regression line.

In [ ]:
# Solution
cov_ms <- cov(students$Math, students$Science)
cor_ms <- cor(students$Math, students$Science)
cat('Covariance =', cov_ms, '\n')
cat('Pearson correlation =', cor_ms, '\n')

# Scatter with regression line
plot(students$Math, students$Science, pch=19, xlab='Math', ylab='Science', main='Math vs Science')
abline(lm(Science ~ Math, data=students), col='red', lwd=2)

# Interpretation:
# - Correlation close to 1/-1 indicates strong linear relationship.
# - Covariance sign indicates direction but is scale-dependent.


## Exercise 3: t-tests
**Problem:** Perform:
1. One-sample t-test to check if mean Math score differs from 70.
2. Two-sample t-test to compare Math scores between Male and Female (assume unequal variance).
3. Paired t-test example: create pre and post scores and test difference.

In [ ]:
# Solution
# 1. One-sample t-test
one_sample <- t.test(students$Math, mu=70)
print(one_sample)
cat('\nInterpretation: If p-value < 0.05, reject H0 that mean = 70.\n')

# 2. Two-sample t-test (unequal variance)
math_male <- subset(students, Gender=='Male')$Math
math_female <- subset(students, Gender=='Female')$Math
two_sample <- t.test(math_male, math_female, var.equal=FALSE)
print(two_sample)
cat('\nInterpretation: p-value < 0.05 indicates significant difference between groups.\n')

# 3. Paired t-test (example)
pre <- round(rnorm(20, 65, 8))
post <- pre + sample(c(-2,0,2,5), 20, replace=TRUE)
paired_test <- t.test(pre, post, paired=TRUE)
print(paired_test)
cat('\nInterpretation: paired t-test checks mean difference within same subjects.\n')


## Exercise 4: Chi-square Test
**Problem:** Test independence between Gender and Class (categorical variables) using chi-square test.

In [ ]:
# Solution
tbl <- table(students$Gender, students$Class)
print(tbl)
chi <- chisq.test(tbl)
print(chi)
cat('\nInterpretation: p-value < 0.05 suggests association between variables.\n')


## Exercise 5: ANOVA
**Problem:** Use one-way ANOVA to check if Math scores differ across Classes A, B, C.

In [ ]:
# Solution
anova_res <- aov(Math ~ Class, data=students)
summary(anova_res)
cat('\nInterpretation: If p-value < 0.05, at least one class mean differs. Use TukeyHSD for pairwise comparisons.\n')

# Pairwise
print(TukeyHSD(anova_res))


## Exercise 6: Simple Linear Regression
**Problem:** Fit a linear model predicting Science from Math. Show coefficients and R-squared. Interpret slope.

In [ ]:
# Solution
lm_fit <- lm(Science ~ Math, data=students)
summary(lm_fit)

# Interpretation:
# - Coefficient for Math is slope: expected change in Science for 1 unit change in Math.
# - R-squared indicates proportion of variance explained by Math.


## Exercise 7: Non-parametric Test (Wilcoxon)
**Problem:** If data is not normally distributed, use Wilcoxon rank-sum test to compare Math scores between two groups.

In [ ]:
# Solution
wilcox_res <- wilcox.test(math_male, math_female)
print(wilcox_res)
cat('\nInterpretation: p-value < 0.05 indicates difference between groups without assuming normality.\n')


## Exercise 8: Grouped Statistics (dplyr) & Visualization (ggplot2) — Optional
**Problem:** Use `dplyr` to compute mean Math by Class and `ggplot2` to plot boxplots of Math by Class.

In [ ]:
# Solution
if(!require(dplyr)) install.packages('dplyr', repos='https://cloud.r-project.org')
if(!require(ggplot2)) install.packages('ggplot2', repos='https://cloud.r-project.org')
library(dplyr)
library(ggplot2)

students %>% group_by(Class) %>% summarise(mean_math = mean(Math), sd_math = sd(Math))

# Boxplot
ggplot(students, aes(x=Class, y=Math)) + geom_boxplot() + labs(title='Math Scores by Class')
